In [1]:
#imports
from collections import Counter  
import pandas as pd
import numpy as np
from collections import defaultdict
#plotting tools
import seaborn as sn
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from copy import deepcopy
import random

#stats module
from scipy import stats

#MOL2VEC related imports
import re, gc
from gensim.models import Word2Vec 
from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec
from gensim.models import word2vec

#rdkit
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit import Chem
from rdkit.Chem import AllChem
from imblearn.under_sampling import RandomUnderSampler 


#ML models and splits from sklearn
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_recall_fscore_support,  accuracy_score, make_scorer


In [2]:
# The default xlrd engine has dropped support for xlsx files
# REF.:https://stackoverflow.com/questions/65254535/xlrd-biffh-xlrderror-excel-xlsx-file-not-supported
xls = pd.ExcelFile('dataset_rdkit.xlsx', engine = 'openpyxl')
df1 = pd.read_excel(xls, 'passb1_commontargets')
df2 = pd.read_excel(xls, 'passb2_commontargets')
df3 = pd.read_excel(xls, 'fail_common_targets')
df = pd.concat([df1,df2, df3])
print(len(df))

3280


In [9]:
## Helper functions 
smi2vec_model = Word2Vec.load('word2vec.model')

#wrong ones dict is a dict used to correct a couple of SMI strings.
wrong_ones_dict = {
    '[H][N]([H])([H])[Pt](Cl)(Cl)[N]([H])([H])[H]' : 'C1CN2C(=NN=C2C(F)(F)F)CN1C(=O)CC(CC3=CC(=C(C=C3F)F)F)N',
    '[H][N]([H])([H])[Pt]1(OC(=O)C2(CCC2)C(=O)O1)[N]([H])([H])[H]': 'C1CC(C1)(C(=O)O)C(=O)O.[NH2-].[NH2-].[Pt+2]',
    '[H][N]1([H])[C@@H]2CCCC[C@H]2[N]([H])([H])[Pt]11OC(=O)C(=O)O1' : 'C1CCC(C(C1)[NH-])[NH-].C(=O)(C(=O)[O-])[O-].[Pt+4]'
}

def smi2mol(x):
    """
    A function that converts SMI strings into RDKit molecules.
    """
    if x in wrong_ones_dict:
        return Chem.MolFromSmiles(wrong_ones_dict[x])
    return Chem.MolFromSmiles(x)

def smiles2vec(x):
    """
    Generates Smi2Vec vectors from a SMI string.
    """
    lst = []
    for word in re.findall(r'.{3}',x):
        if word in smi2vec_model.wv:
            lst.append(smi2vec_model.wv[word])
    return sum(lst)/len(lst)

def get_maccs_from_mol(mol):
    """
    Generates MACCS vectors from a SMI string.
    """
    return np.array(MACCSkeys.GenMACCSKeys(mol))

def read_sheets(sheet_list):
    sheet_list = [pd.read_excel(xls, x) for x in sheet_list]
    return pd.concat(sheet_list)

def get_hyperparams(X, y, scoring = None):
    classifiers1 = [
        GridSearchCV(RandomForestClassifier(n_jobs = -1), {'n_estimators':[100, 200,300,250],'criterion':["gini", "entropy"],'max_depth':[10,15,20, None]}, n_jobs  =-1, verbose = 3, scoring = scoring),
        GridSearchCV(ExtraTreesClassifier(n_jobs = -1),{'n_estimators':[100, 200,300,250],'criterion':["gini", "entropy"],'max_depth':[10,15,20, None]}, n_jobs  =-1, verbose = 3, scoring = scoring),
        GridSearchCV(AdaBoostClassifier(),{'n_estimators':[100, 200,300,250]}, n_jobs  =-1, verbose = 3, scoring = scoring),
        GridSearchCV(SVC(), {'kernel':['rbf'], 'gamma':["scale"] }, n_jobs  =-1, verbose = 3, scoring = scoring),
        GridSearchCV(LogisticRegression(), {'penalty' : ['l2', 'none']}, n_jobs  =-1, scoring = scoring),
        GridSearchCV(KNeighborsClassifier(), {'n_neighbors': list(range(10)), 'weights':["distance", "uniform"]}, n_jobs  =-1, verbose = 3, scoring = scoring),
        GridSearchCV(GaussianNB(), {'var_smoothing' : [1e-9]}, n_jobs  =-1, verbose = 3, scoring = scoring), 
        GridSearchCV(MLPClassifier(), {'hidden_layer_sizes' : [[100], [100, 200], [100, 150, 200]], "learning_rate_init":[0.01, 0.001, 0.05]}, n_jobs  =-1, verbose = 3, scoring = scoring)
    ]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    classifiers1 = [x.fit(X_scaled, y).best_params_ for x in classifiers1]
    return classifiers1

def get_classifiers(classifiers1):
    classifiers = [
            #{0:1, 1:5}
            RandomForestClassifier(n_jobs = -1, **classifiers1[0],class_weight="balanced"),
            ExtraTreesClassifier(n_jobs = -1, **classifiers1[1],class_weight="balanced"),
            #GradientBoostingClassifier(**classifiers1[2]),
            AdaBoostClassifier(**classifiers1[2]),
            SVC(**classifiers1[3],class_weight="balanced"),
            LogisticRegression(**classifiers1[4],class_weight="balanced"),
            KNeighborsClassifier(**classifiers1[5]),
            GaussianNB(**classifiers1[6]),
            MLPClassifier(**classifiers1[7])
        ]
    return classifiers

def get_classifiers_weighted(classifiers1):
    classifiers = [
            RandomForestClassifier(n_jobs = -1, **classifiers1[0],class_weight={1:5, 0:1}),
            ExtraTreesClassifier(n_jobs = -1, **classifiers1[1],class_weight={1:5, 0:1}),
            AdaBoostClassifier(**classifiers1[2]),
            SVC(**classifiers1[3],class_weight={1:5, 0:1}),
            LogisticRegression(**classifiers1[4],class_weight={1:5, 0:1}),
            KNeighborsClassifier(**classifiers1[5]),
            GaussianNB(**classifiers1[6]),
            MLPClassifier(**classifiers1[7])
        ]
    return classifiers

def get_basic_classifiers():
    classifiers = [
            RandomForestClassifier(n_jobs = -1),
            ExtraTreesClassifier(n_jobs = -1),
            AdaBoostClassifier(),
            SVC(),
            LogisticRegression(),
            KNeighborsClassifier(),
            GaussianNB(),
            MLPClassifier()
        ]
    return classifiers

def get_train_test_split(X, y, split_indices):
    X1_test_set = []
    X1_test_set.extend(split_indices[1])
    X1_test_mask = np.zeros((X.shape[0],), bool)
    X1_test_mask[X1_test_set] = True
    X1_test = X[X1_test_mask]
    X1 = X[~X1_test_mask]
    y1 = y[~X1_test_mask]
    y1_test = y[X1_test_mask]
    
    return X1, X1_test, y1, y1_test

def f1_fail_score(gt, pred):
    p0, r0, f0, _ = precision_recall_fscore_support(gt, pred, pos_label = 1, average = "binary")
    return f0

In [4]:
# Fingerprint functions
# accept a dataframe with the 'smi' field, and returns a numpy array of fingerprints for each row of the dataframe.
def mol2vec(df):
    df = df.copy(deep = True)
    df['mol'] = df['smi'].apply(smi2mol)
    model = word2vec.Word2Vec.load('../model_300dim.pkl')
    #Constructing sentences
    df['sentence'] = df.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)
    #Extracting embeddings to a numpy.array
    #Note that we always should mark unseen='UNK' in sentence2vec() so that model is taught how to handle unknown substructures
    df['mol2vec'] = [DfVec(x) for x in sentences2vec(df['sentence'], model, unseen='UNK')]
    X_mol2vec = np.array([x.vec for x in df['mol2vec']])
    return X_mol2vec

def morgan(df):
    df = df.copy(deep = True)
    X_mol2vec = np.array([AllChem.GetMorganFingerprintAsBitVect(smi2mol(x),2,1024) for x in df['smi']])
    return X_mol2vec

def mol2vec_ft(df):
    df = df.copy(deep = True)
    df['mol'] = df['smi'].apply(smi2mol)
    model = word2vec.Word2Vec.load('../updatedmodel.pkl')
    #Constructing sentences
    df['sentence'] = df.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)

    #Extracting embeddings to a numpy.array
    #Note that we always should mark unseen='UNK' in sentence2vec() so that model is taught how to handle unknown substructures
    df['mol2vec'] = [DfVec(x) for x in sentences2vec(df['sentence'], model, unseen='UNK')]
    X_mol2vec = np.array([x.vec for x in df['mol2vec']])
    return X_mol2vec

def smi2vec(df):
    df = df.copy(deep = True)
    df['smi2vec'] = df["smi"].apply(lambda x : smiles2vec(x))
    X_smi2vec = np.array([x for x in df['smi2vec']])
    return X_smi2vec
def maccs(df):
    df = df.copy(deep = True)
    df['mol'] = df['smi'].apply(smi2mol)
    maccs_keys = df['mol'].apply(get_maccs_from_mol)
    maccs_keys = maccs_keys.to_numpy()
    maccs_keys = np.asarray(list(maccs_keys))
    return maccs_keys
    
## Testing code for all the fingerpring modules
for fp in [mol2vec, mol2vec_ft, smi2vec, maccs, morgan]:
    x = fp(df)
    print(fp.__name__, x.shape)

RDKit WARNING: [00:44:19] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [00:44:19] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [00:44:19] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [00:44:19] WARNING: not removing hydrogen atom without neighbors


mol2vec (3280, 300)


RDKit WARNING: [00:44:24] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [00:44:24] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [00:44:24] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [00:44:24] WARNING: not removing hydrogen atom without neighbors


mol2vec_ft (3280, 300)
smi2vec (3280, 100)


RDKit WARNING: [00:44:28] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [00:44:28] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [00:44:28] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [00:44:28] WARNING: not removing hydrogen atom without neighbors


maccs (3280, 167)


RDKit WARNING: [00:44:34] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [00:44:34] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [00:44:34] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [00:44:34] WARNING: not removing hydrogen atom without neighbors


morgan (3280, 1024)


In [5]:
# Experiment details
sheets = {
    'main' : ['passb1_commontargets', 'passb2_commontargets', 'fail_common_targets', "p2fail_commontargets"],
    'ki' : ['passb1_common_ki', 'passb2_common_ki', 'fail_common_ki', "p2fail_common_ki"],
    'stdval' : ['passb1_common_stdval', 'passb2_common_stdval', 'fail_common_stdval', "p2fail_common_stdval"],
    'main_v1' : ['passb1_commontargets', 'passb2_commontargets', 'fail_common_targets'],
    'ki_v1' : ['passb1_common_ki', 'passb2_common_ki', 'fail_common_ki'],
    'stdval_v1' : ['passb1_common_stdval', 'passb2_common_stdval', 'fail_common_stdval'],
}

#fingerprints to be appended to the datasets using these functions
fingerprints = {
    'mol2vec': mol2vec,
    'mol2vec_ft' : mol2vec_ft,
    'smi2vec' : smi2vec,
    'maccs' : maccs
}

#base features to be included in all experiments
base_features = ["label","smi",'mol_w', 'num_valence_electrons', 'num_heteroatoms', 'gastiger_charges', 'tpsa', 'h_acceptors',\
             'h_donors', 'n_rotatable_bonds', 'n_rings', 'num_of_atoms', 'num_of_heavy_atoms', "count_drug_targets", "count_of_approved_trials", "count_common_targets"]

#Uncomment below for running on subset of features
# base_features = ["label","smi",'mol_w', 'num_valence_electrons', 'num_heteroatoms', 'gastiger_charges', 'tpsa', 'h_acceptors',\
#              'h_donors', 'n_rotatable_bonds', 'n_rings', 'num_of_atoms', 'num_of_heavy_atoms', "count_drug_targets", "count_common_targets"]

experiment = {'data' : 'stdval_v1',
              'fp' : mol2vec, 
              'additional_features' : ['medianstdval', 'maxstdval', 'minsdtval'], 
              'n_clusters' : 6}

#uncomment this statement for experiments on ki data.
# experiment = {'data' : 'ki_v1',
#               'fp' : mol2vec, 
#               'additional_features' : ["minki"], 
#               'n_clusters' : 6}

#List of classifiers used in the experiment.
names = ["RF", "ExtRa Trees",  "Adaboost","SVM", "Log. Reg.", "kNN" , "Naive Bayes", "NN"]

In [6]:
final_output_csv = None

#output csv for this experiment, will contain result of each fold.
output_csv = {'intervention':[], 'train_size' : [], 'test_size' : [], 'confusion' : [],\
              'auc' : []}
for i in "aprf":
    for j in names:
        output_csv[i+"_"+j] = []
        if i != 'a':
            output_csv[i+"0_"+j] = []

# read the data
df = read_sheets(sheets[experiment['data']])
# Drop NaN values from required rows
df = df.loc[:, df.columns.isin(base_features+ experiment['additional_features'] + ["minsdtval", "minki", "drugbank id", "count_common_targets"])].dropna()
df = df.loc[:, df.columns.isin(base_features+ experiment['additional_features'] + ["drugbank id", "count_common_targets"])].dropna()
# Reset the index. This is crucial since we will use indices for splits
df.reset_index(drop=True, inplace=True)

# Filter out the data to have common targets <= 6
df = df[df['count_common_targets'] <=6]

experiment['n_passes'] = len(df[df['label'] == 0])
experiment['n_fails'] = len(df[df['label'] == 1])
y = df.loc[:, df.columns.isin(['label'])]
y = y.iloc[:, 0].to_numpy()
colnames = list(df.loc[:, df.columns.isin(base_features [2:] + experiment['additional_features'] + ["drugbank id"])].columns)
X = df.loc[:, df.columns.isin(base_features [2:] + experiment['additional_features'] + ["drugbank id"])].to_numpy()

# standard assertions
assert X.shape[1] == len(base_features[2:] + experiment['additional_features']+ ['drugbank id']), "SOME FEATURES MISSING!!!!"
assert y.shape[0] == X.shape[0], "missing rows!"

# Generate fingerprints in case the experiment requires them.
if experiment["fp"] is not None:
    X = np.concatenate([X, experiment['fp'](df)], 1)
    
print(X.shape, y.shape)    
df = None

# Obtain the optimal hyperparameters for this dataset through GridSearch
classifiers1 = get_hyperparams(X[:,1:], y, scoring = make_scorer(f1_fail_score, greater_is_better=True))

# a set to keep track of processed drugs
leftout_drugbank_ids = set()
# list of pass samples
positive_indices = list(np.where(y==0)[0])

# Leave one out cross validation
for idx in tqdm(list(np.where(y==1)[0])):
    # clear classifiers from previous iteration
    gc.collect()
    
    did_name = X[idx, 0]
    if y[idx]==0:
        continue
    
    for i in range(4):
        mask = X[:,0] == -1
        mask[idx] = True
        mask[random.choice(positive_indices)] = True

        X_train = X[~mask][:,1:]
        y_train = y[~mask]
        X_test = X[mask][:,1:]
        y_test = y[mask]
        
        print(X_train.shape[0], X_test.shape[0])
        classifiers = get_classifiers(classifiers1)

        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        accuracy_values = []
        pvalues = []
        rvalues = []
        fvalues = []

        pvalues0 = []
        rvalues0 = []
        fvalues0 = []

        nmes = []
        logreg_confusion = None
        logreg_auc = None

        for idx, clf in enumerate(classifiers):
            clf.fit(X_train, y_train)
            pred = clf.predict(X_test)
            accuracy_values.append(
                accuracy_score(y_test, pred)
            )
            p, r, f, _ = precision_recall_fscore_support(y_test, pred, pos_label = 0, average = "binary")
            pvalues.append(p)
            rvalues.append(r)
            fvalues.append(f)
            p0, r0, f0, _ = precision_recall_fscore_support(y_test, pred, pos_label = 1, average = "binary")
            pvalues0.append(p0)
            rvalues0.append(r0)
            fvalues0.append(f0)
            nmes.append(names[idx])

            if names[idx] == "Log. Reg.":
                logreg_confusion = confusion_matrix(y_test, pred).__str__()
                pred = clf.predict_proba(X_test)
                try:
                    logreg_auc = roc_auc_score(y_test, pred[:,0])
                except ValueError:
                    logreg_auc = -1
                    
        acc_dict = {
            "a" : accuracy_values,
            "p" : pvalues, 
            "r" : rvalues, 
            "f" : fvalues,
            "p0" : pvalues0, 
            "r0" : rvalues0, 
            "f0" : fvalues0,
            "algorithm" : nmes
        }

        output_csv["confusion"].append(logreg_confusion)
        output_csv["auc"].append(logreg_auc)
        output_csv["intervention"].append(did_name)
        output_csv['train_size'].append(X_train.shape[0])
        output_csv['test_size'].append(X_test.shape[0])
        
        for ii in names:
            for jj in "arpf":
                output_csv[jj+"_"+ii].append(acc_dict[jj][acc_dict["algorithm"].index(ii)])
                if jj != 'a':
                    output_csv[jj+"0_"+ii].append(acc_dict[jj+'0'][acc_dict["algorithm"].index(ii)])


output_csv['intervention'].append("mean")
output_csv['confusion'].append("mean")

for i in output_csv:
    if i not in ["intervention", "confusion"]:
        output_csv[i].append(sum(output_csv[i])/len(output_csv[i]))

output_csv['intervention'].append("std")
output_csv['confusion'].append("std")
for i in output_csv:
    if i not in ["intervention", "confusion"]:
        output_csv[i].append(np.std(np.array(output_csv[i][:-1])))
for i in output_csv:
    if i != "intervention":
        output_csv[i].extend([np.nan]*15)
    else:
        output_csv[i].extend(["data : %s, fp : %s, additional_features : %s, passes, fails : %d, %d"%(experiment['data'], \
                        experiment['fp'].__name__, ",".join(experiment['additional_features'])
                        , experiment['n_passes'], experiment['n_fails'])] + [np.nan]*14)

if final_output_csv is None:
    final_output_csv = deepcopy(output_csv)
else:
    for i in output_csv:
        final_output_csv[i].extend(output_csv[i])
df = pd.DataFrame(final_output_csv)
df.to_csv("loocv_result.csv")

(852, 316) (852,)
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/sas/vidhya/CompoundDb4jV2/conda/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fitting 5 folds for each of 20 candidates, totalling 100 fits


/sas/vidhya/CompoundDb4jV2/conda/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.68438052 0.68438052 0.68438052 0.55500411
 0.70714673 0.52168175 0.71063879 0.2669427  0.72682693 0.31734418
 0.73683977 0.15607407 0.74134427 0.17022792 0.74134427 0.14688517
 0.7346776  0.17250142]
  warnings.warn(


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 9 candidates, totalling 45 fits


...

/sas/vidhya/CompoundDb4jV2/conda/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/sas/vidhya/CompoundDb4jV2/conda/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/sas/vidhya/CompoundDb4jV2/conda/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data 

850 2


/sas/vidhya/CompoundDb4jV2/conda/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/sas/vidhya/CompoundDb4jV2/conda/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/sas/vidhya/CompoundDb4jV2/conda/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data 

    intervention  train_size  test_size        confusion  auc  a_RF  \
0        DB00834       850.0        2.0  [[0 1]\n [1 0]]  1.0   0.5   
1        DB00834       850.0        2.0  [[0 1]\n [1 0]]  1.0   0.5   
2        DB00834       850.0        2.0  [[0 1]\n [1 0]]  1.0   0.5   
3        DB00834       850.0        2.0  [[0 1]\n [1 0]]  1.0   0.5   
4        DB00413       850.0        2.0  [[0 1]\n [1 0]]  1.0   0.5   
..           ...         ...        ...              ...  ...   ...   
436          NaN         NaN        NaN              NaN  NaN   NaN   
437          NaN         NaN        NaN              NaN  NaN   NaN   
438          NaN         NaN        NaN              NaN  NaN   NaN   
439          NaN         NaN        NaN              NaN  NaN   NaN   
440          NaN         NaN        NaN              NaN  NaN   NaN   

     a_ExtRa Trees  a_Adaboost  a_SVM  a_Log. Reg.  ...  f_SVM  f0_SVM  \
0              0.5         0.5    0.0          0.0  ...    0.0     0.0   

/sas/vidhya/CompoundDb4jV2/conda/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
